# Pipeline

1. Preprocessing
2. Data Extraction
3. Data Exploration
4. **Model**

This file initializes the model and makes predictions too.

# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
# Get feature dataframe from pickle file
df = pd.read_pickle('feature_frame.pkl')

In [12]:
df.sample(5)

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,REL_PTS_1_away,REL_PTS_2_away,REL_PTS_3_away,REL_PTS_4_away,REL_PTS_5_away,REL_PTS_N-0_away,REL_PTS_N-1_away,REL_PTS_N-2_away,REL_PTS_N-3_away,REL_PTS_N-4_away
1397,13,21,1,1,0,2014-02-12,0.394737,0.421053,0.078947,0.342105,...,1.115385,1.083077,1.076923,0.961538,0.846154,-0.230769,-0.153846,-0.116923,-0.115385,-0.038462
557,22,5,0,2,2,2011-12-27,0.500000,0.250000,0.111111,0.444444,...,0.264706,0.264706,0.000000,-0.346405,-0.352941,-1.568627,-1.457516,-1.346405,-1.294118,-1.235294
347,8,17,3,0,1,2011-05-01,0.647059,0.235294,0.235294,0.176471,...,0.914286,0.828571,0.742857,0.616450,0.446218,-0.200000,-0.171429,-0.171429,-0.085714,-0.057143
4640,51,49,2,1,1,2014-05-10,0.636364,0.181818,0.470588,0.176471,...,1.500000,0.941176,0.701426,0.647059,0.580214,-0.359180,-0.328877,-0.205882,-0.088235,-0.088235
8772,82,71,3,2,1,2017-04-22,0.473684,0.263158,0.307692,0.307692,...,1.439394,1.189394,1.126894,0.848485,0.845644,-0.560606,-0.404356,-0.248106,-0.123106,0.000000


# Extracting target variables and features

# _WICHTIG: Das hier muss auf neue columns angepasst werden (z.B. season)_

In [32]:
# If True, the goal difference for a specific game is seen as the target variable.
# (e.g. -3 for a game outcome of 1:4, or 2 for 3:1) .
#
# If False, we just want to predict the winner.
# 1 = Home team wins, 0 = Draw, 2 = Away team wins
predict_goal_difference = False

if predict_goal_difference:
    y = df['FTHG'] - df['FTAG']
else:
    y = df['FTR']
    
# Remove unnecessary columns (IDs etc.) from features
X = df.iloc[:,6:].drop(['season', 'GAME_CNT_AFTER_GAME_home', 'GAME_CNT_AFTER_GAME_away', 'league'], axis=1)    

In [ ]:
df.iloc[:,6:].drop(['season', 'GAME_CNT_AFTER_GAME_home', 'GAME_CNT_AFTER_GAME_away'], axis=1).columns

# Ranked Probability Score (RPS)

In [24]:
def rps(pred, actual_value, r=3):
    '''Returns the ranked probability score for a single given game.
    (see Hubacek paper for formula)
    
    Arguments:
    pred -- predicted results; in vector form (e.g. [0.1, 0.6, 0.3])
    actual_value -- actual result (0, 1 or 2); not in vector form yet
    r -- number of categories (3 for football)
    '''
    value_vec = [0, 0, 0]
    
    # Bring value_vec into 1, 0, 2 order
    if actual_value == 0:
        value_vec[1] = 1
    elif actual_value == 2:
        value_vec[0] == 1
    elif actual_value == 1:
        value_vec[2] == 1
    else:
        raise Exception('Prediction was not in [1, 0, 2].')
    #value_vec = [0, 0, 1]
    
    #pred[0], pred[1] = pred[1], pred[0]
    
    pred[0], pred[1], pred[2] = pred[2], pred[0], pred[1]   # order: loss, draw, win
    
    #print(pred)
    
    rps = 0
    
    for i in range(0, r-1):    # r-1 becomes r because of the exclusion of range()
        bracket_part = 0
        for j in range(0, i+1):    # same for i and i+1
            bracket_part += pred[j] - value_vec[j]
        
        rps += np.square(bracket_part)
    
    rps *= 1 / (r - 1)
    
    return rps


# To be used as eval_metric parameter
def rps_eval_metric(y_true, y_pred):
    return rps(y_pred, y_true)

# Model fit

In [55]:
# Apparently the random states are pretty important. 21 works very well on RPS and test accuracy, 16 only on accuracy.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.222, random_state=20)

## Find best value for *n_estimators*

In [ ]:
# We start with n_estimators=50...
xgb_cl = xgb.XGBClassifier(objective='multi:softmax', n_estimators=50, seed=16)
# ...and then validate it to find the lowest loss.
op = xgb_cl.fit(X_train, y_train, early_stopping_rounds=900, eval_metric='mlogloss', eval_set=[(X_test, y_test)])

In [24]:
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
n_estimators = [4, 8, 16, 32]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)

In [26]:
# I tried to do Grid Search here, but the 

grid = GridSearchCV(estimator=xgb_cl, param_grid=param_grid, n_jobs=8, cv=3)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:29:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.475635 using {'learning_rate': 0.2, 'n_estimators': 8}


## Use best parameters to fit the model

In [56]:
# n_estimators=8 had the lowest loss, so we overwrite the previous model.b
xgb_cl = xgb.XGBClassifier(objective='multi:softprob', n_estimators=8, seed=16, learning_rate=0.3, eval_metric=rps_eval_metric)

In [57]:
xgb_cl.fit(X_train, y_train)

d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:04:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              eval_metric=<function rps_eval_metric at 0x000001E22331BC10>,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=8, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', random_state=16,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, seed=16,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

# Evaluation

In [58]:
preds = xgb_cl.predict(X_test)
accuracy = float(np.sum(preds == y_test))/y_test.shape[0]

print(f'accuracy: {accuracy}')

accuracy: 0.6344755970924195


In [59]:
# Looking at train set accuracy to get an intuition of how much the model overfits
preds = xgb_cl.predict(X_train)
accuracy = float(np.sum(preds == y_train))/y_train.shape[0]

print(f'accuracy on train set: {accuracy}')

accuracy on train set: 0.6718364121307913


In [60]:
# Predictions in form of [0.2, 0.5, 0.3] in the order of draw, home win, away win.
# This is because it gets ordered like 0, 1, 2.
proba_preds = xgb_cl.predict_proba(X_test)

# List of RPS scores for every game in the test set.
# Important to use iloc for y_test, otherwise indices would be wrong
rps_list = [rps(pred, y_test.iloc[i]) for i, pred in enumerate(proba_preds)]

# Average ranked probability score.
np.mean(rps_list)

0.21778030378136548